## Importing Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
movie = pd.read_csv("movies.csv")
rating = pd.read_csv("ratings.csv")

## Analysing the dataset

In [ ]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movie = movie.iloc[:,0:-1]

In [ ]:
movie.isnull().sum()

movieId    0
title      0
dtype: int64

In [ ]:
movie.duplicated().sum()

0

In [ ]:
rating.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
rating.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [ ]:
rating.duplicated().sum()

0

In [ ]:
rating = rating.iloc[:,0:-1]

In [ ]:
movie.shape

(9742, 2)

In [ ]:
rating.shape

(100836, 3)

In [ ]:
no_of_ratings_for_each_movie = rating.groupby('movieId').count()['userId'].reset_index()
no_of_ratings_for_each_movie.rename(columns={'userId':'No. of ratings'},inplace=True)
no_of_ratings_for_each_movie.head()

,movieId,No. of ratings
0,1,215
1,2,110
2,3,52
3,4,7
4,5,49


In [ ]:
avg_rating_for_each_movie=rating.groupby('movieId').mean()['rating'].reset_index()
avg_rating_for_each_movie.rename(columns={'rating':'Avg. Rating'},inplace=True)
avg_rating_for_each_movie.head()

,movieId,Avg. Rating
0,1,3.920930
1,2,3.431818
2,3,3.259615
3,4,2.357143
4,5,3.071429


In [ ]:
movie_info = no_of_ratings_for_each_movie.merge(avg_rating_for_each_movie,on='movieId')
movie_info.head()

,movieId,No. of ratings,Avg. Rating
0,1,215,3.920930
1,2,110,3.431818
2,3,52,3.259615
3,4,7,2.357143
4,5,49,3.071429


## Colaborative Model

In [ ]:
pt = movie.merge(rating, on='movieId')
pt.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [ ]:
for_similar_movies = pt.pivot_table(index='title', columns='userId', values = 'rating')
for_similar_movies.fillna(0,inplace=True)
for_similar_movies.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for_similar_movies.shape

(9719, 610)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(for_similar_movies)

In [ ]:
def knn_recommend(movie_name , k=5):
  index = np.where(for_similar_movies.index == movie_name)[0][0]   # getting the index of the movie which i want to find similar books for
  similar_movies = similarity_matrix[index]                        # getting the similarity scores for that movie
  top_similar_indices_of_movies = np.argsort(similar_movies)[::-1][:k+1]  #gettig the top k similar movies, we take k+1 similar movies as the 1st movie (with max similarity) will be the movie itself
  top_similar_movies = for_similar_movies.index[top_similar_indices_of_movies]  #  # Getting the index of the top k similar movies from the 'for_similar_movies' table
  return top_similar_movies.tolist()

In [ ]:
user_movie = input("Enter the name of the movie for which you want to find similar movies: ")
no_of_movies = int(input("Enter how many similar movies you want to watch: "))

Enter the name of the movie for which you want to find similar movies: Toy Story (1995)
Enter how many similar movies you want to watch: 3


In [ ]:
recommend_movies = knn_recommend(user_movie,no_of_movies)
recommend_movies.pop(0)               # as the 1st movie is the movie itself so we dont need to include that in the similar movies. So we remove that movie
print("Movies similar to {} are: \n".format(user_movie))
for movie in recommend_movies:
  print(movie)

Movies similar to Toy Story (1995) are: 

Toy Story 2 (1999)
Jurassic Park (1993)
Independence Day (a.k.a. ID4) (1996)


## Genre Based Recommendation

In [ ]:
movie2 = pd.read_csv('movies.csv')

In [ ]:
movie2.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [ ]:
movie2['genres'] = movie2['genres'].str.replace('|',' ')      # to separate the different genres by a space instead of a '|'.
movie2.head(1)

<ipython-input-25-645a66863c0e>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movie2['genres'] = movie2['genres'].str.replace('|',' ')      # to separate the different genres by a space instead of a '|'.


,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 30, stop_words = 'english')
vector = cv.fit_transform(movie2['genres']).toarray()
vector.shape

(9742, 23)

In [ ]:
print(vector)

[[0 1 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
# To visualize what the feature extracted vector looks like any movie
movie_list = movie2['title'].tolist()  # As 'title' is the column containing movie names
for title, movie_vector in zip(movie_list[:5], vector[:5]):     # i am doing movie_list[:5] sp that only the 1st five movies and their features are printed
    print(f"Movie: {title}, Features: {movie_vector}")

Movie: Toy Story (1995), Features: [0 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Movie: Jumanji (1995), Features: [0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Movie: Grumpier Old Men (1995), Features: [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Movie: Waiting to Exhale (1995), Features: [0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Movie: Father of the Bride Part II (1995), Features: [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


This finds the similarity between the rows , i.e similarity between the different movies based on genre.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_between_eachMovies_genres = cosine_similarity(vector)
print(similarity_between_eachMovies_genres)

[[1.         0.77459667 0.31622777 ... 0.         0.31622777 0.4472136 ]
 [0.77459667 1.         0.         ... 0.         0.         0.        ]
 [0.31622777 0.         1.         ... 0.         0.         0.70710678]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.31622777 0.         0.         ... 0.         1.         0.        ]
 [0.4472136  0.         0.70710678 ... 0.         0.         1.        ]]


In [ ]:
def recommend(movie, k):
    index = np.where(movie2['title'] == movie)[0][0]
    similar_movies = similarity_between_eachMovies_genres[index]
    # print(similar_movies.shape)        # this will print (9742,1)
    top_k_indices = np.argsort(similar_movies)[::-1][:k+1]
    top_movies = movie2['title'].iloc[top_k_indices]
    return top_movies.tolist()

# def recommend(movie,k):
#   index = np.where(movie2.index == movie)[0][0]
#   print(index)
#   similar_movies = similarity_between_eachMovies_genres[index]
#   top_k_indices = np.argsort(similar_movies)[::-1][:k]
#   top_movies = movie2.index[top_k_indices]

#   return top_movies.tolist()



Break down the expression np.argsort(similar_movies)[::-1][:k+1] step by step:

np.argsort(similar_movies): The function np.argsort() returns the indices that would sort the array similar_movies in ascending order. In this case, it will give the indices that sort similar_movies in increasing order of similarity scores.

[::-1]: The slice notation [::-1] is used to reverse the order of the array obtained from np.argsort(). This results in an array of indices sorted in descending order of similarity scores.

[:k+1]: Finally, we use slicing to select the first k+1 elements from the reversed array of indices. The k+1 elements are taken because we want to include the first k most similar movies along with the movie itself in the recommendations.

The code line top_movies = movie2['title'].iloc[top_k_indices] selects the movie titles from the 'title' column of the DataFrame movie2 corresponding to the indices stored in the top_k_indices array. As a result, top_movies will be a pandas Series containing the titles of the movies that are considered the most similar based on their genre similarity scores.

In [ ]:
user_movie = input("Enter the movie for which you want to find similar movies: ")
no_of_movies = int(input("Enter how many similar movies do you want to know: "))

Enter the movie for which you want to find similar movies: 10 Things I Hate About You (1999)
Enter how many similar movies do you want to know: 3


In [ ]:
movies_which_are_similar = recommend(user_movie, no_of_movies)
movies_which_are_similar.pop(0)
print("Movies with similar genre to {} are: \n".format(user_movie))
for movie in movies_which_are_similar:
  print(movie)

Movies similar to 10 Things I Hate About You (1999) are: 

Down with Love (2003)
Playing It Cool (2014)
Bachelor, The (1999)
